## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Amahai,ID,-3.3333,128.9167,81.63,74,62,2.84,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.52,77,43,12.30,light rain
2,2,Leningradskiy,RU,69.3833,178.4167,34.32,94,26,9.71,scattered clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,29.86,93,75,5.75,broken clouds
4,4,Cidreira,BR,-30.1811,-50.2056,65.35,89,51,9.51,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please input the min temp for your trip. "))
max_temp = float(input("Please input the max temp for your trip. "))

Please input the min temp for your trip. 75
Please input the max temp for your trip. 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Amahai,ID,-3.3333,128.9167,81.63,74,62,2.84,broken clouds
6,6,Weligama,LK,5.9667,80.4167,78.46,88,100,11.10,overcast clouds
8,8,Butaritari,KI,3.0707,172.7902,82.87,68,100,4.68,overcast clouds
9,9,Ailigandi,PA,9.2333,-78.0167,80.73,88,100,6.69,overcast clouds
11,11,Calabozo,VE,8.9242,-67.4293,77.04,85,55,2.66,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID            0
City               0
Country            0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna()
pref_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Amahai,ID,-3.3333,128.9167,81.63,74,62,2.84,broken clouds
6,6,Weligama,LK,5.9667,80.4167,78.46,88,100,11.10,overcast clouds
8,8,Butaritari,KI,3.0707,172.7902,82.87,68,100,4.68,overcast clouds
9,9,Ailigandi,PA,9.2333,-78.0167,80.73,88,100,6.69,overcast clouds
11,11,Calabozo,VE,8.9242,-67.4293,77.04,85,55,2.66,broken clouds
...,...,...,...,...,...,...,...,...,...,...
674,674,Jiexiu,CN,37.0244,111.9125,78.80,18,0,5.50,clear sky
677,677,Batagay,RU,67.6333,134.6333,81.32,28,100,21.99,overcast clouds
678,678,Ambon,ID,-3.6954,128.1814,78.94,94,75,12.39,broken clouds
682,682,Pinar Del Rio,CU,22.4175,-83.6981,78.03,80,69,1.43,broken clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Amahai,ID,81.63,broken clouds,-3.3333,128.9167,
6,Weligama,LK,78.46,overcast clouds,5.9667,80.4167,
8,Butaritari,KI,82.87,overcast clouds,3.0707,172.7902,
9,Ailigandi,PA,80.73,overcast clouds,9.2333,-78.0167,
11,Calabozo,VE,77.04,broken clouds,8.9242,-67.4293,
13,The Valley,AI,81.10,overcast clouds,18.2170,-63.0578,
14,Isangel,VU,82.51,few clouds,-19.5500,169.2667,
21,Atuona,PF,79.02,light rain,-9.8000,-139.0333,
22,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,
23,Avarua,CK,78.85,moderate rain,-21.2078,-159.7750,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    print(f"Processing record {index}")
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("No Hotel .. Skipping")        

Processing record 0
Processing record 6
Processing record 8
Processing record 9
Processing record 11
Processing record 13
Processing record 14
Processing record 21
Processing record 22
Processing record 23
Processing record 24
Processing record 31
Processing record 35
No Hotel .. Skipping
Processing record 44
Processing record 46
Processing record 50
Processing record 51
Processing record 52
Processing record 59
Processing record 63
Processing record 65
Processing record 68
Processing record 69
Processing record 70
Processing record 72
Processing record 73
Processing record 74
Processing record 78
Processing record 85
Processing record 86
Processing record 91
Processing record 97
Processing record 105
Processing record 108
Processing record 113
Processing record 114
Processing record 118
Processing record 120
Processing record 125
No Hotel .. Skipping
Processing record 130
Processing record 135
Processing record 136
Processing record 137
Processing record 142
Processing record 144
Proc

In [11]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Amahai,ID,81.63,broken clouds,-3.3333,128.9167,HOTEL ISABELA
6,Weligama,LK,78.46,overcast clouds,5.9667,80.4167,Weligama Bay Resort
8,Butaritari,KI,82.87,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
9,Ailigandi,PA,80.73,overcast clouds,9.2333,-78.0167,Akwa Reef Lodge
11,Calabozo,VE,77.04,broken clouds,8.9242,-67.4293,Hotel Giardini
...,...,...,...,...,...,...,...
674,Jiexiu,CN,78.80,clear sky,37.0244,111.9125,IU Hotel Jinzhong Yuhua Road
677,Batagay,RU,81.32,overcast clouds,67.6333,134.6333,Gostinitsa Ki'ileekh
678,Ambon,ID,78.94,broken clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
682,Pinar Del Rio,CU,78.03,broken clouds,22.4175,-83.6981,Hostal La Colina Dunay y Raul


In [12]:
# 7. Drop the rows where there is no Hotel Name.

# Replacing the empty string to Null values.
hotel_df.loc[(hotel_df["Hotel Name"]==""),"Hotel Name"]= np.nan

# Drop the rows with no Hotel Name
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Amahai,ID,81.63,broken clouds,-3.3333,128.9167,HOTEL ISABELA
6,Weligama,LK,78.46,overcast clouds,5.9667,80.4167,Weligama Bay Resort
8,Butaritari,KI,82.87,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
9,Ailigandi,PA,80.73,overcast clouds,9.2333,-78.0167,Akwa Reef Lodge
11,Calabozo,VE,77.04,broken clouds,8.9242,-67.4293,Hotel Giardini
...,...,...,...,...,...,...,...
674,Jiexiu,CN,78.80,clear sky,37.0244,111.9125,IU Hotel Jinzhong Yuhua Road
677,Batagay,RU,81.32,overcast clouds,67.6333,134.6333,Gostinitsa Ki'ileekh
678,Ambon,ID,78.94,broken clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
682,Pinar Del Rio,CU,78.03,broken clouds,22.4175,-83.6981,Hostal La Colina Dunay y Raul


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPY_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weaher Description</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} *F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))